In [2]:
import os
from bseflow import model_variations
import subprocess
import re

In [3]:
%load_ext autoreload
%autoreload 2

Create tasks for intermediate stage rates per formation channel (Classic CE, OSMT, Single-core CE, Double-core CE)

In [4]:
## Write disbatch tasks to tasks directory on cluster

run_dir = "/mnt/home/alam1/ceph/disBatch_dir/bseflow"

if not os.path.isdir(run_dir):
    print('run_dir = ', run_dir)
    os.makedirs(run_dir, exist_ok=True)
else:
    print(f'Nothing to do, {run_dir} already exists')

model_files = [model_variations[f'{variation}']['file'] for variation in model_variations.keys()]

channels = ['SMTCE', 'OSMT', 'SingleCore', 'DoubleCore']
channel_flags = [('OS_pre_SN', 'CE_post_SN'), ('OS_pre_SN', 'OS_post_SN'), 'single_core', 'double_core']


for i, channel in enumerate(channels):
    with open(f'{run_dir}/{channel}task', 'w') as f:
        start_line = f"cd {run_dir} ; module load python disBatch ; \n"
        f.write(start_line)
        for model in model_files:
            if isinstance(channel_flags[i], tuple): 
                channel_args = f'--MT1mask {channel_flags[i][0]} --MT2mask {channel_flags[i][1]}'
            else:
                channel_args = f'--MT1mask {channel_flags[i]}'
            additional_args = f'--save_path {channel} --output_dir bseflow'
            task_line = f"python3 /mnt/home/alam1/bseflow/bseflow/output_rates.py /mnt/home/alam1/ceph/xfer/{model}/COMPASOutput.h5 {channel_args} {additional_args}"
            f.write(task_line + '\n')     

Nothing to do, /mnt/home/alam1/ceph/disBatch_dir/bseflow already exists


In [ ]:
# disBatch Command (-t 1-0 is also fine)

for i, channel in enumerate(channels):
    command = f"cd {run_dir} && module load disBatch && sbatch -t 0-16:00:00 -p gen -n 18 disBatch {channel}task"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Extract the job ID from the output
    match = re.search(r"Submitted batch job (\d+)", result.stdout)
    if match:
        disBatch_job_ids = match.group(1)

    print(disBatch_job_ids)

4659129


Create tasks for intermediate stage rates for complete simulations (for Sankey results)

In [13]:
with open(f'{run_dir}/alltask', 'w') as f:
    start_line = f"cd {run_dir} ; module load python disBatch ; \n"
    f.write(start_line)
    for model in model_files:
        args = f'--save_path all --output_dir bseflow'
        task_line = f"python3 /mnt/home/alam1/bseflow/bseflow/output_rates.py /mnt/home/alam1/ceph/xfer/{model}/COMPASOutput.h5 {args}"
        f.write(task_line + '\n') 